In [2]:
import pandas as pd
import numpy as np
import os
import re
import sys
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import argparse

In [29]:


def label_regex (x) :
    x = str(x)
    x = re.sub('중성화수술','Neutral', x)
    x = re.sub('중성화 수술','Neutral', x)
    x = re.sub('중성화','Neutral', x)
    x = re.sub('여아중성화','Neutral', x)
    x = re.sub('남아중성화','Neutral', x)
    x = re.sub('여아Neutral','Neutral', x)#여아Neutral
    x = re.sub('남아Neutral','Neutral', x)
    x = re.sub('여중','Neutral', x)
    x = re.sub('남중','Neutral', x)

    x = re.sub('[건강][가-힣 ()a-zA-Z,0-9\/]+','test', x)

    x = re.sub('X','normal', x)
    x = re.sub('x','normal', x)
    x = re.sub('이상없음','normal', x)
    x = re.sub('XX','normal', x)
    x = re.sub('-','normal', x)
    x = re.sub('normalnormal','normal', x)
    x = re.sub('정상','normal', x)
    x = re.sub('없음','normal', x)

    x = re.sub('스켈링','스케일링', x)

    x = re.sub('[.]0','', x)
    x = re.sub('[ ]+','', x)

    x = re.sub('[(].+[)]', '', x)
    return x

# df['진단코드'] = label_regex(df['진단코드'])
def my_strip_split (df) :
    for column in ['보호자명', '환자명', '생일', '검사일', '병원'] :
        df[column] = df[column].apply(lambda x : str(x).strip())
    for column in ['생일', '검사일'] :
        df[column] = df[column].apply(lambda x : x.split(' ')[0])
    for column in ['병원'] :
        df[column] = df[column].apply(lambda x : x.split('_')[0])
    return df

def csv_concat(data_path) :

    dfall = []
    #temp = [x.decode('euc-kr') for x in os.listdir(data_path)]
    # print(os.listdir(data_path))
    dir_list = [data_path+ '/' + d for d in os.listdir(data_path) if os.path.isdir(data_path + '/' + d)]
#    print(dir_list)
    for d in dir_list :
        print(d)
#         files = [f for f in os.listdir(d) if f.endswith('코드적용.xlsx')]
        files = [f for f in os.listdir(d)]
        print(files)
        for f in files :
#            f = d + '/' + f
            print(f)
            df = pd.read_excel(d + '/' + f)
            if f.startswith('D01') :
                df = pd.read_excel(d + '/' + f, 'Sheet1')
#             print(df.columns)
            df['병원'] = d.split('/')[1]
            df1 = df[['보호자명', '환자명', '품종코드','생일', '성별코드', '검사일', '병원']]
            if '주진단명\n코드 1' in df.columns :
                temp = df[['주진단명\n코드 1','기타진단명\n코드 2', '기타진단명\n코드3', '기타진단명코드 4', '기타진단명코드 5']]
                df1['진단코드'] = temp[temp.columns[:]].apply(
                    lambda x: ','.join(x.dropna().astype(str)),
                    axis=1)
                df1.rename(columns ={'주진단명\n코드 1':'주진단코드'}, inplace=True)

            elif '주진단명코드 1' in df.columns :
                temp = df[['주진단명코드 1', '기타진단명코드 2', '기타진단명코드3', '기타진단명코드 4', '기타진단명코드 5']]
                df1['진단코드'] = temp[temp.columns[:]].apply(
                    lambda x: ','.join(x.dropna().astype(str)),
                    axis=1)
                df1.rename(columns ={'주진단명코드 1':'주진단코드'}, inplace=True)
            elif '주진단명코드\n(단일항목기재)' in df.columns :
                temp = df[['주진단명코드\n(단일항목기재)', '기타진단명코드\n(복수기재가능)']]
                df1['진단코드'] = temp[temp.columns[:]].apply(
                    lambda x: ','.join(x.dropna().astype(str)),
                    axis=1)
                df1.rename(columns ={'주진단명코드\n(단일항목기재)':'주진단코드'}, inplace=True)
            df1['SE'] = df['SE']
            dfall.append(df1)
    df = pd.concat(dfall, ignore_index=True)
    return df

def make_num_label(df) :
    df['Num_Label'] = ''
    df.Num_Label = df['진단코드'].apply(lambda x : len(str(x).split(',')))
    return df

#not used
def make_label_dict(df, Num_class=50) :
    for labels in df['진단코드'] :
        labels = str(labels).split(',')
        for label in labels:
            la = str(label).split('/')
            for l in la :
                l = label_regex(l)
                if l in label_dict :
                    label_dict[l] += 1
                else :
                    label_dict[l] = 1
    label_sorted = sorted(label_dict.items(), key=(lambda x : x[1]), reverse=True)
    label_sorted_ = label_sorted[0:Num_class]
    label_sorted_ = [label for label, number in label_sorted_]

def make_label_col(df, label_list) :
    for index, labels in zip(df.index, df['진단코드']) :
        labels = str(labels).split(',')
        for label in labels :
            la = str(label).split('/')
            for l in la :
                l = label_regex(l)
                if l in label_list :
                    df.loc[index, l] = 1
                else :
                    df.loc[index, l] = 0
    return df

def set_label(x, label_list) :
    if x in label_list :
        return x
    else :
        return 'others'

def mak_label_id(df) :
    encoder = LabelEncoder()
    encoder.fit(df.label)
    label_id = encoder.transform(df.label)
    df['label_id'] = label_id
    return df




In [30]:
def main(data_set, saved_dir, num=50):

    df = csv_concat(data_set)
    print('total df shape : ',df.shape)
    df = df.dropna(subset=['SE']).reset_index(drop=True)
    print('drop SE shape : ',df.shape)
    df = my_strip_split(df)
    df['진단코드'] = df['진단코드'].apply(lambda x : np.nan if str(x).strip(' \t') == '' else x)
    df = df.dropna(subset=['진단코드']).reset_index(drop=True)
    print('drop diagnoise code shape : ',df.shape)
    df['진단코드'] = df['진단코드'].apply(lambda x : label_regex(x))
    df['label'] = df['진단코드'].apply(lambda x : str(x).split(',')[0])
#    print(df.label.value_counts())

    labels = df.label.value_counts()[0:num].to_dict()
    label_list = [label for label in labels]

    df.label = df.label.apply(lambda x : set_label(str(x), label_list))

    label_dict = {}
    print(df.columns)
#    for index, label in zip(range(len(df.columns[1:])), df.columns[1:]) :
    for index, label in zip(range(len(label_list)), label_list) :
        label_dict[label] = index
    label_dict['others'] = len(label_list)

    df['label_id'] = df['label'].apply(lambda x : label_dict[str(x)])

    if not (os.path.isdir(saved_dir)) :
        os.makedirs(saved_dir)

    df.to_csv(saved_dir + '/' + 'df.csv', index=None)

    train_df, test_df = train_test_split(df, test_size = 0.1, random_state=100)
    print('train_len {} test_len {}'.format(len(train_df), len(test_df)))
    train_df.to_csv(saved_dir + '/' + 'train.csv', index=None)
    test_df.to_csv(saved_dir + '/' + 'test.csv', index=None)


In [31]:
main('data', 'files2')


data/D
['D01.xlsx', 'D02.xlsx']
D01.xlsx


/home/beomgon/bert/bert/bert-env/lib/python3.6/site-packages/ipykernel_launcher.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/beomgon/bert/bert/bert-env/lib/python3.6/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/beomgon/bert/bert/bert-env/lib/python3.6/site-packages/ipykernel_launcher.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.

D02.xlsx
data/C
['C02.xlsx', 'C01.xlsx']
C02.xlsx
C01.xlsx
data/A
['A02.xlsx', 'A01.xlsx']
A02.xlsx


/home/beomgon/bert/bert/bert-env/lib/python3.6/site-packages/ipykernel_launcher.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


A01.xlsx


/home/beomgon/bert/bert/bert-env/lib/python3.6/site-packages/ipykernel_launcher.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


data/F
['F01.xlsx']
F01.xlsx
data/B
['B01.xlsx', 'B02.xlsx']
B01.xlsx
B02.xlsx
data/E
['E01.xlsx']
E01.xlsx
data/G
['G01.xlsx']
G01.xlsx
total df shape :  (16459, 9)
drop SE shape :  (14992, 9)
drop diagnoise code shape :  (13366, 9)
Index(['보호자명', '환자명', '품종코드', '생일', '성별코드', '검사일', '병원', '진단코드', 'SE',
       'label'],
      dtype='object')
train_len 12029 test_len 1337


In [4]:
# import argparse
# import os
# import sklearn
# import argparse
# import tensorflow as tf
# import pandas as pd
# import numpy as np
# from keras.preprocessing.sequence import pad_sequences

# import sys
# sys.path.append('../ALBERT')
# import time
# import modeling
# import tokenization
# import optimization



# def get_class_num(f) :
#     df = pd.read_csv(f)
#     return len(df.label_id.value_counts())

# def process_input(file, mode, tokenizer):
# #    df_data = pd.read_csv(file, names = ['SE','label_id'], skiprows =1)
#     df_data = pd.read_csv(file)
#     df_data = df_data[['SE', 'label_id']]
#     df_data.dropna(subset=['SE'], inplace=True)
#     print('mode ', mode)

#     if ((mode == tf.estimator.ModeKeys.TRAIN)|(mode == tf.estimator.ModeKeys.EVAL)):

#         df_data['text_tokens'] = df_data.SE.apply(tokenizer.tokenize)
#         print(df_data['text_tokens'])
# #         print(([df_data.text_tokens]))
# #         print(len(df_data.text_tokens))

# #         input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(list(df_data.text_tokens))],
# #                           maxlen=args.max_seq_len, dtype="long", truncating="post", padding="post")
#         input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in df_data['text_tokens']],
#                           maxlen=args.max_seq_len, dtype="long", truncating="post", padding="post")

#         labels = df_data.label_id
#         # create the mask to ignore the padded elements in the sequences.
#         input_mask = [[int(i>0) for i in ii] for ii in input_ids]
#         return input_ids, labels, input_mask

#     else: # Predict
#         df_data['text_tokens'] = df_data.SE.apply(tokenizer.tokenize)
#         input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in df_data['text_tokens']],
#                           maxlen=args.max_seq_len, dtype="long", truncating="post", padding="post")
#         # create the mask to ignore the padded elements in the sequences.
#         input_mask = [[int(i>0) for i in ii] for ii in input_ids]
#         #print(df_data['text_tokens'])
#         #print(input_ids)
#         return input_ids, None, input_mask

# def input_fn_builder(file, seq_length, drop_remainder, tokenizer):
#     """Creates an `input_fn` closure to be passed to TPUEstimator."""

#     def input_fn(mode, params):
#         print("mode:", mode)

#         input_ids, labels, input_mask = process_input(file, mode, tokenizer)
#         print("input_ids : ", input_ids[0:2] )
# #        print("labels : ", labels )
#         print("input_mask : ", input_mask[0:2] )

#         """The actual input function."""
#         batch_size = params["batch_size"]

#         num_examples = len(input_ids)
#         print("******num_examples******", num_examples)

#         # This is for demo purposes and does NOT scale to large data sets. We do
#         # not use Dataset.from_generator() because that uses tf.py_func which is
#         # not TPU compatible. The right way to load data is with TFRecordReader.
#         if ((mode == tf.estimator.ModeKeys.TRAIN)|(mode == tf.estimator.ModeKeys.EVAL)):
#             d = tf.data.Dataset.from_tensor_slices(
#                 ({"input_ids": tf.constant( input_ids, shape=[num_examples, seq_length], dtype=tf.int32),
#                   "input_mask": tf.constant( input_mask, shape=[num_examples, seq_length], dtype=tf.int32),
#                   "segment_ids": tf.zeros(shape=[num_examples, seq_length], dtype=tf.int32),}
#                  ,tf.constant(labels, shape=[num_examples], dtype=tf.int32),))

#             if mode == tf.estimator.ModeKeys.TRAIN:
#                 d = d.repeat()
#                 d = d.shuffle(buffer_size=100)

#             d = d.batch(batch_size=batch_size, drop_remainder=drop_remainder)
#             return d
#         else: # Predict
#             d = tf.data.Dataset.from_tensor_slices(
#                 ({"input_ids": tf.constant( input_ids, shape=[num_examples, seq_length], dtype=tf.int32),
#                   "input_mask": tf.constant( input_mask, shape=[num_examples, seq_length], dtype=tf.int32),
#                   "segment_ids": tf.zeros(shape=[num_examples, seq_length], dtype=tf.int32),}
#                  ,tf.zeros(shape=[num_examples], dtype=tf.int32),))

#             d = d.batch(batch_size=batch_size, drop_remainder=drop_remainder)
#             return d

#     return input_fn

# def create_model(albert_config, mode, input_ids, input_mask, segment_ids,
#                                  labels, num_labels):
#     """Creates a classification model."""
#     is_training = mode == tf.estimator.ModeKeys.TRAIN
#     model = modeling.AlbertModel(
#             config=albert_config,
#             is_training=is_training,
#             input_ids=input_ids,
#             input_mask=input_mask,
#             token_type_ids=segment_ids)

#     #output_layer = model.get_sequence_output()
#     output_layer = model.get_pooled_output()

#     hidden_size = output_layer.shape[-1].value
#     print("*************************num_labels***********", num_labels)

#     output_weight = tf.get_variable(
#             "output_weights", [num_labels, hidden_size],
#             initializer=tf.truncated_normal_initializer(stddev=0.02))

#     output_bias = tf.get_variable(
#             "output_bias", [num_labels], initializer=tf.zeros_initializer())

#     with tf.variable_scope("loss"):
#         if mode == tf.estimator.ModeKeys.TRAIN:
#             output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)
#         output_layer = tf.reshape(output_layer, [-1, hidden_size])
#         logits = tf.matmul(output_layer, output_weight, transpose_b=True)
#         logits = tf.nn.bias_add(logits, output_bias)
#         #logits = tf.reshape(logits, [-1, args.max_seq_len, num_labels])
#         logits = tf.reshape(logits, [-1, num_labels])
#         probabilities = tf.nn.softmax(logits, axis=-1)
#         predict = tf.argmax(probabilities,axis=-1)
#         if ((mode == tf.estimator.ModeKeys.TRAIN)|(mode == tf.estimator.ModeKeys.EVAL)):
#             log_probs = tf.nn.log_softmax(logits, axis=-1)
#             one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)
#             per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
#             loss = tf.reduce_sum(per_example_loss)
#             return (loss, per_example_loss, logits,predict)
#         else: # Predict
#             return (None, None, logits,predict)


# def model_fn_builder(albert_config, num_labels, init_checkpoint, learning_rate,
#                      num_train_steps, num_warmup_steps, use_tpu,
#                      hub_module=None, optimizer="adamw"):
#     """Returns `model_fn` closure for TPUEstimator."""

#     def model_fn(features, labels, mode, params):       # pylint: disable=unused-argument
#         """The `model_fn` for TPUEstimator."""

#         tf.logging.info("*** Features ***")
#         for name in sorted(features.keys()):
#             tf.logging.info(f"  name = {name}, shape = {features[name].shape}")

#         input_ids = features["input_ids"]
#         input_mask = features["input_mask"]
#         segment_ids = features["segment_ids"]
#         label_ids = labels

#         (total_loss, per_example_loss, logits, predictions) = \
#                 create_model(albert_config, mode, input_ids, input_mask,
#                                          segment_ids, label_ids, num_labels)

#         tvars = tf.trainable_variables()
#         initialized_variable_names = {}
#         scaffold_fn = None
#         if init_checkpoint:
#             (assignment_map, initialized_variable_names
#             ) = modeling.get_assignment_map_from_checkpoint(tvars, init_checkpoint)
#             if use_tpu:
#                 def tpu_scaffold():
#                     tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
#                     return tf.train.Scaffold()

#                 scaffold_fn = tpu_scaffold
#             else:
#                 print("********weights is initialize from checkpoint**********")
#                 tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

#         tf.logging.info("**** Trainable Variables ****")
#         for var in tvars:
#             init_string = ""
#             if var.name in initialized_variable_names:
#                 init_string = ", *INIT_FROM_CKPT*"
#             tf.logging.info(f"  name = {var.name}, shape = {var.shape}{init_string}")

#         output_spec = None
#         if mode == tf.estimator.ModeKeys.TRAIN:

#             train_op = optimization.create_optimizer(
#                     total_loss, learning_rate, num_train_steps, num_warmup_steps,
#                     use_tpu, optimizer)

#             output_spec = tf.contrib.tpu.TPUEstimatorSpec(
#                     mode=mode,
#                     loss=total_loss,
#                     train_op=train_op,
#                     scaffold_fn=scaffold_fn)
#         elif mode == tf.estimator.ModeKeys.EVAL:
#             def metric_fn(per_example_loss, label_ids, logits):
#                 predictions = tf.argmax(logits, axis=-1, output_type=tf.int32)
#                 accuracy = tf.metrics.mean(tf.math.equal(label_ids,predictions))
#                 loss = tf.metrics.mean(values=per_example_loss)
#                 #
#                 return {
#                     "eval_accuracy":accuracy,
#                     "eval_loss": loss,
#                 }
#             eval_metrics = (metric_fn, [per_example_loss, label_ids, logits])
#             output_spec = tf.contrib.tpu.TPUEstimatorSpec(
#                     mode=mode,
#                     loss=total_loss,
#                     eval_metrics=eval_metrics,
#                     scaffold_fn=scaffold_fn)
#         else: # Predict
#             output_spec = tf.contrib.tpu.TPUEstimatorSpec(
#                     mode=mode,
#                     predictions= predictions,
#                     scaffold_fn=scaffold_fn)
#         return output_spec
#     return model_fn
        
    
# def get_accuracy(predictions, file) :
#     df_test = pd.read_csv(file)
#     labels = list(df_test.label_id)

#     right = 0
#     wrong = 0

#     for i in range(len(labels)) :
#         if labels[i] == predictions[i] :
#             right += 1
#         else :
#             wrong += 1
#             print( "Wrong label : {} predict {} ".format(labels[i], predictions[i]) )
#     accuracy = right/(right+wrong)
# #    print('accuracy ', accuracy)
#     return accuracy
    

In [12]:
# def train_main(args) :

#     Num_class = get_class_num(args.train_files)
#     print("Num Class : ", Num_class)
#     print(args)

#     config = modeling.AlbertConfig.from_json_file('albert_config.json')
#     tokenizer = tokenization.FullTokenizer.from_scratch(vocab_file = args.vocab_files, do_lower_case=False, spm_model_file=None)

#     print("fine tune dir", args.finetune)
#     USE_TPU = False
# #     run_config = tf.contrib.tpu.RunConfig(model_dir=args.finetune,)
#     run_config = tf.contrib.tpu.RunConfig(model_dir='finetune3',)
    
#     model_fn = model_fn_builder(
#             albert_config=config,
# #            num_labels=len(label_list),
#             num_labels=Num_class,
#             init_checkpoint=args.init_checkpoint,
#             learning_rate=args.rate,
#             num_train_steps=args.num_train_step,
#             num_warmup_steps=args.warmup_step,
#             use_tpu=USE_TPU)

#     # If TPU is not available, this will fall back to normal Estimator on CPU
#     # or GPU.
#     estimator = tf.contrib.tpu.TPUEstimator(
#             use_tpu=USE_TPU,
#             model_fn=model_fn,
#             config=run_config,
#             train_batch_size=args.batchsize,
#             eval_batch_size=args.batchsize,
#             predict_batch_size=args.batchsize)

# #     tf.logging.info("***** Running training *****")
# #     # import sys
# #     start = time.time()

# #     train_input_fn = input_fn_builder(
# #             file = args.train_files,
# #             tokenizer = tokenizer,
# #             seq_length=args.max_seq_len,
# #             drop_remainder=True)

# #     estimator.train(input_fn=train_input_fn, max_steps=args.num_train_step)

# #     print("training time :", time.time() - start)


# # #     tf.saved_model.save(model_fn, args.saved_path)

# #     predict_input_fn = input_fn_builder(
# #             file = args.test_files,
# #             tokenizer = tokenizer,
# #             seq_length=args.max_seq_len,
# #             drop_remainder=False)


# #     predictions = []
# #     print('finetune3' + '/model.ckpt-' + str(args.num_train_step))
# # #     for prediction in estimator.predict(input_fn=predict_input_fn, checkpoint_path=args.finetune + '/model.ckpt-' + str(args.num_train_step)):
# #     for prediction in estimator.predict(input_fn=predict_input_fn, checkpoint_path='finetune3' + '/model.ckpt-' + str(args.num_train_step)):        
# #         #print("*******prediction*********", prediction)
# #         predictions.append(prediction)

# #     accuracy = get_accuracy(predictions, args.test_files)
# #     print('test accuracy ', accuracy)



In [33]:
# parser = argparse.ArgumentParser()
# parser.add_argument("-tr", "--train_files", help="train csv file", default="files2/train.csv")
# parser.add_argument("-te", "--test_files", help="test csv file", default="files2/test.csv")
# parser.add_argument("-c", "--config_files", help="config file", default="albert_config.json")
# parser.add_argument("-v", "--vocab_files", help="vocab file", default="vocab.txt")
# parser.add_argument("-s", "--saved_path", help="saved path", default="saved/")
# # parser.add_argument("-e", "--epochs", help="epochs", type=int, default=1)
# parser.add_argument("-b", "--batchsize", help="batchsize", type=int, default=16)
# parser.add_argument("-r", "--rate", help="learning_rate", type=float, default=0.0001)
# parser.add_argument("-w", "--warmup_step", help="warmup steps", type=int, default=1500)
# parser.add_argument("-n", "--num_train_step", help="num train steps", type=int, default=3000)
# parser.add_argument("-i", "--init_checkpoint", help="checkpoint init dir", default="models/")
# parser.add_argument("-f", "--finetune", help="fine tunning dir", default="finetune3/")
# parser.add_argument("-m", "--max_seq_len", help="max_seq_len", type=int,  default=250)
# #    parser.add_argument("-w", "--weights", help="weights", action="store_true")
# #    parser.add_argument("-t", "--trainable", help="trainable", action="store_true")
# #    parser.add_argument("-c", "--checkpoint", help="checkpoint")
# args = parser.parse_args()
# print(args)
# args.finetune_dir = "finetune3"
# print(args.finetune_dir)
# train_main(args)


In [100]:
df = pd.read_csv('files2/df.csv')

In [101]:
df.head(10)

,보호자명,환자명,품종코드,생일,성별코드,검사일,병원,진단코드,SE,label,label_id
0,김소야*7,코코,91.0,2001-05-03,MN,2018-07-04,D,"2078,만성신질환,신장결석,방광염","S) 식욕부진, 기력저하, 눈 O) - 2일동안 식욕이 거의 없어서 보호자분...",others,50
1,이지은*6,힘찬이,119.0,2007-06-04,MN,2018-04-26,D,"2244,담낭점액종","S) 다음다뇨, 식욕부진 O) - 최근들어 다음다뇨 증상과 2~3일전부터는 ...",2244,26
2,홍성미,비쥬,499.0,2004-01-01,FS,2018-07-20,D,"2082,만성신질환,부비동염","[오전 통화 by 관중] - 일주일동안 밥안먹고, 활력저하됨 - 보호자님 복부촉...",2082,4
3,김래경,세순이,107.0,2001-08-01,FS,2017-06-16,D,"2087,단백뇨,쿠싱,빈혈",1. 쿠싱약 안먹고 와서 쿠싱검사는 28일에 하기로 함 2. 흉방에서 CVC 허탈...,2087,6
4,김지현*6,효,125.0,2009-11-05,MN,2018-02-28,D,"2081,방광결석","S) 건강검진 진행함 O) 1. 혈액검사 - 간수치 상승(ALP, ALT ...",2081,34
5,박수진,티키,598.0,2005-05-04,FS,2017-10-08,D,"2244,지방간,췌장염","538,300원 선결제 하셨습니다. - 송이 S) 식욕부진, 구토 O) ...",2244,26
6,서주현,돼지,130.0,2004-03-08,MN,2018-02-14,D,"2231,방광결석,간비대","S) 체중감소, 건강검진 O) - 혈액검사 : 빈혈 진행중(33%) - 염...",others,50
7,오주영,달콩이,128.0,2004-06-06,MN,2018-04-21,D,"2177,갑기저,고혈압,방광염,쿠싱","667,060 미수금 발생 - 휘린 [한방진료] -후지 근육 강화를 위한 ...",others,50
8,정여연*7,유자,91.0,2009-10-28,FS,2018-02-11,D,2092,내추럴발란스캔 결제완료후 두고 가심 18.02.12일에 가져가실예정 효정 ...,2092,10
9,강보람,콩쥐,125.0,2005-03-31,FS,2018-06-19,D,2196,S) - 특이사항 없음. - 잘 지냄. O) - 청진 시 NRF -...,others,50


In [102]:
# import pandas as pd
# file_dir = 'files2'
# df = pd.read_csv(file_dir + "/df.csv")
# Num_Class = len(df.label.value_counts())
# print(Num_Class)
# df.head()
# print(df.shape)

In [103]:
# df.label.value_counts()[1:20].to_dict()
label_list_20 = []
for key in df.label.value_counts()[1:20].to_dict():
    label_list_20.append(key)
print(label_list_20)
df = df[df.loc[:,'label']!='others']
print(df.shape)

['normal', 'Neutral', '2046', '2043', '2082', '2001', '2087', '2101', '2071', '2170', '2092', '2275', '2116', '2133', 'test', '2185', '2095', '2126', '2048']
(9623, 11)


In [104]:
df.label = df.label.apply(lambda x : x if str(x) in label_list_20 else 'others' )
df.label.value_counts()
print(df.head(10))

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(df.label)
X_train_encoded = encoder.transform(df.label)
df['label_id'] =  X_train_encoded
df.head(10)

     보호자명  환자명   품종코드          생일 성별코드         검사일 병원             진단코드  \
1   이지은*6  힘찬이  119.0  2007-06-04   MN  2018-04-26  D       2244,담낭점액종   
2     홍성미   비쥬  499.0  2004-01-01   FS  2018-07-20  D  2082,만성신질환,부비동염   
3     김래경  세순이  107.0  2001-08-01   FS  2017-06-16  D   2087,단백뇨,쿠싱,빈혈   
4   김지현*6    효  125.0  2009-11-05   MN  2018-02-28  D        2081,방광결석   
5     박수진   티키  598.0  2005-05-04   FS  2017-10-08  D     2244,지방간,췌장염   
8   정여연*7   유자   91.0  2009-10-28   FS  2018-02-11  D             2092   
10    송혜림   쿠키  131.0  2011-03-01    F  2018-03-18  D        2091,기도허탈   
11  문은진*7   다온  119.0  2011-04-18   FS  2018-02-17  D             2091   
12    박해춘  박두식  598.0  2011-06-02   MN  2018-04-20  D          2087,빈혈   
13  박예지*6   보리  119.0  2011-06-02   FS  2018-02-23  D             2170   

                                                   SE   label  label_id  
1   S) 다음다뇨, 식욕부진    O)  - 최근들어 다음다뇨 증상과 2~3일전부터는 ...  others        26  
2   [오전 통화 by 관중]  - 일주일동안 밥안먹고, 활력저하

,보호자명,환자명,품종코드,생일,성별코드,검사일,병원,진단코드,SE,label,label_id
1,이지은*6,힘찬이,119.0,2007-06-04,MN,2018-04-26,D,"2244,담낭점액종","S) 다음다뇨, 식욕부진 O) - 최근들어 다음다뇨 증상과 2~3일전부터는 ...",others,18
2,홍성미,비쥬,499.0,2004-01-01,FS,2018-07-20,D,"2082,만성신질환,부비동염","[오전 통화 by 관중] - 일주일동안 밥안먹고, 활력저하됨 - 보호자님 복부촉...",2082,5
3,김래경,세순이,107.0,2001-08-01,FS,2017-06-16,D,"2087,단백뇨,쿠싱,빈혈",1. 쿠싱약 안먹고 와서 쿠싱검사는 28일에 하기로 함 2. 흉방에서 CVC 허탈...,2087,6
4,김지현*6,효,125.0,2009-11-05,MN,2018-02-28,D,"2081,방광결석","S) 건강검진 진행함 O) 1. 혈액검사 - 간수치 상승(ALP, ALT ...",others,18
5,박수진,티키,598.0,2005-05-04,FS,2017-10-08,D,"2244,지방간,췌장염","538,300원 선결제 하셨습니다. - 송이 S) 식욕부진, 구토 O) ...",others,18
8,정여연*7,유자,91.0,2009-10-28,FS,2018-02-11,D,2092,내추럴발란스캔 결제완료후 두고 가심 18.02.12일에 가져가실예정 효정 ...,2092,7
10,송혜림,쿠키,131.0,2011-03-01,F,2018-03-18,D,"2091,기도허탈",1.CC : 혈뇨 2.HPI - 4~5일 전에 혈뇨 증상 처음 발견( C...,others,18
11,문은진*7,다온,119.0,2011-04-18,FS,2018-02-17,D,2091,cc: 건강검진 S) - 어제 자기 전 오심 한 것 외에 특이사항없음 - ...,others,18
12,박해춘,박두식,598.0,2011-06-02,MN,2018-04-20,D,"2087,빈혈",S) - 체중도 찌고 상태는 양호함. - 설사는 지속되고 있음. 정상변-설사 ...,2087,6
13,박예지*6,보리,119.0,2011-06-02,FS,2018-02-23,D,2170,"cc: 건강검진 S) - 꼬리부분, 복부 육아직 재 확인 요청 - 식탐 많...",2170,13


In [173]:
# from sklearn import preprocessing
# le = preprocessing.LabelEncoder()
# le.fit(data['name'])
le_name_mapping = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))
print(le_name_mapping)


{'2001': 0, '2043': 1, '2046': 2, '2048': 3, '2071': 4, '2082': 5, '2087': 6, '2092': 7, '2095': 8, '2101': 9, '2116': 10, '2126': 11, '2133': 12, '2170': 13, '2185': 14, '2275': 15, 'Neutral': 16, 'normal': 17, 'others': 18, 'test': 19}


In [106]:
df.tail()

,보호자명,환자명,품종코드,생일,성별코드,검사일,병원,진단코드,SE,label,label_id
13360,김옥,뽀뽀(R),125.0,2014-11-12,MN,2018-11-12,G,normal,리셉션(외래진료) - 주증상 : 슬개골탈구 재발 - 증상발현일 : 처음 슬...,normal,17
13362,이순옥,다솔(R),125.0,2013-11-12,FS,2018-11-13,G,2082,<오전관리사항> Food - 식사종류 / 양 : 금식 - 식욕상태 ...,2082,5
13363,장윤경,아키,128.0,2012-11-13,MN,2018-11-13,G,2046,S) - 조금전 음식물 쓰레기 뒤져 닭뼈 다량 섭식. - 구토했는데 조각 일부 ...,2046,2
13364,박남선,이쁜이(R),131.0,2009-11-13,FS,2018-11-22,G,normal,"우측 CCRL 수술/ 양측서혜탈장 수술 <수술설명> - 연구개노장, 기관...",normal,17
13365,한순옥,루비,125.0,2011-11-10,F,2018-11-14,G,2274,동거견에 물려 안구돌출 보호자분 요청으로 병리학적 검사 진행 O) 혈액검사 상...,others,18


In [107]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df,test_size=0.1, random_state=100 )
train.to_csv('files3/train.csv', index=None)
test.to_csv('files3/test.csv', index=None)
print(train.shape)
print(test.shape)

(8660, 11)
(963, 11)


In [1]:
import pandas as pd
file_dir = 'files'
df = pd.read_csv(file_dir + "/train.csv")
Num_Class = len(df.label.value_counts())
print(Num_Class)
df.head()
print(df.shape)
del df

20
(8660, 11)


In [2]:
import sys
sys.path.append('../ALBERT')
import tensorflow as tf
import modeling
import tokenization
import optimization
import pandas as pd
import numpy as np
from keras.preprocessing.sequence import pad_sequences
tf.enable_eager_execution()
import tensorflow as tf

Using TensorFlow backend.


In [37]:
MAX_LEN = 250 #95 percentile of validated top relevant text length is 50
BATCH_SIZE = 16
MAX_GRAD_NORM = 1.0
LEARNING_RATE = 0.0001
NUM_WARMUP_STEPS= 10000
OUTPUT_DIR = "finetune1"
USE_TPU = False
num_class = Num_Class

tf.logging.set_verbosity(tf.logging.INFO)
label_list = [i for i in range(num_class)]
init_checkpoint="models/"
num_train_steps = 100000 
run_config = tf.contrib.tpu.RunConfig(model_dir=OUTPUT_DIR, keep_checkpoint_max=60, save_checkpoints_secs=900,)

In [38]:
config = modeling.AlbertConfig.from_json_file("albert_config.json")
tokenizer = tokenization.FullTokenizer.from_scratch(vocab_file="vocab.txt", do_lower_case=False, spm_model_file=None)
# Test on tokenizer
print(tokenizer.tokenize("안녕 반가워 내이름은 범곤, 나는 라면을 좋아해 !! "))

['안', '##녕', '반', '##가', '##워', '내', '##이', '##름', '##은', '범', '##곤', ',', '나는', '라', '##면', '##을', '좋아', '##해', '!', '!']


In [39]:

def process_input(file, mode, tokenizer):
    df_data = pd.read_csv(file)
    df_data = df_data[['SE', 'label_id']]
#     print(df.columns)
#     df_data = pd.read_csv(file, names = ['SE','label_id'], skiprows =1)
    df_data = df_data.dropna(subset=['SE'])

    if ((mode == tf.estimator.ModeKeys.TRAIN)|(mode == tf.estimator.ModeKeys.EVAL)):    

        df_data['text_tokens'] = df_data.SE.apply(tokenizer.tokenize)
        print(([df_data.text_tokens]))
#         print(len(df_data.text_tokens))

#         input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(list(df_data.text_tokens))],
#                           maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
        input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in df_data['text_tokens']],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
        
        labels = df_data.label_id
        # create the mask to ignore the padded elements in the sequences.
        input_mask = [[int(i>0) for i in ii] for ii in input_ids]
        return input_ids, labels, input_mask

    else: # Predict 
        df_data['text_tokens'] = df_data.SE.apply(tokenizer.tokenize)
        input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in df_data['text_tokens']],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
        # create the mask to ignore the padded elements in the sequences.
        input_mask = [[int(i>0) for i in ii] for ii in input_ids]
        #print(df_data['text_tokens'])
        #print(input_ids)
        return input_ids, None, input_mask

In [40]:
def input_fn_builder(file, seq_length, drop_remainder, tokenizer):
    """Creates an `input_fn` closure to be passed to TPUEstimator."""

    def input_fn(mode, params):
        print("mode:", mode)
        
        input_ids, labels, input_mask = process_input(file, mode, tokenizer)
        print("input_ids : ", input_ids[0:5] )
#         print("labels : ", labels )
#         print("input_mask : ", input_mask[0:5] )
    
        """The actual input function."""
        batch_size = params["batch_size"]

        num_examples = len(input_ids)
        print("******num_examples******", num_examples)

        # This is for demo purposes and does NOT scale to large data sets. We do
        # not use Dataset.from_generator() because that uses tf.py_func which is
        # not TPU compatible. The right way to load data is with TFRecordReader.
        if ((mode == tf.estimator.ModeKeys.TRAIN)|(mode == tf.estimator.ModeKeys.EVAL)):
#             d = tf.data.Dataset.from_tensor_slices(
#                 ({"input_ids": tf.constant( input_ids, shape=[num_examples, seq_length], dtype=tf.int32),
#                   "input_mask": tf.constant( input_mask, shape=[num_examples, seq_length], dtype=tf.int32),
#                   "segment_ids": tf.zeros(shape=[num_examples, seq_length], dtype=tf.int32),}
#                  ,tf.constant(labels, shape=[num_examples, seq_length], dtype=tf.int32),))
            d = tf.data.Dataset.from_tensor_slices(
                ({"input_ids": tf.constant( input_ids, shape=[num_examples, seq_length], dtype=tf.int32),
                  "input_mask": tf.constant( input_mask, shape=[num_examples, seq_length], dtype=tf.int32),
                  "segment_ids": tf.zeros(shape=[num_examples, seq_length], dtype=tf.int32),}
                 ,tf.constant(labels, shape=[num_examples], dtype=tf.int32),))            

            if mode == tf.estimator.ModeKeys.TRAIN:
                d = d.repeat()
                d = d.shuffle(buffer_size=100)

            d = d.batch(batch_size=batch_size, drop_remainder=drop_remainder)
            return d
        else: # Predict
#             d = tf.data.Dataset.from_tensor_slices(
#                 ({"input_ids": tf.constant( input_ids, shape=[num_examples, seq_length], dtype=tf.int32),
#                   "input_mask": tf.constant( input_mask, shape=[num_examples, seq_length], dtype=tf.int32),
#                   "segment_ids": tf.zeros(shape=[num_examples, seq_length], dtype=tf.int32),}
#                  ,tf.zeros(shape=[num_examples, seq_length], dtype=tf.int32),))
            d = tf.data.Dataset.from_tensor_slices(
                ({"input_ids": tf.constant( input_ids, shape=[num_examples, seq_length], dtype=tf.int32),
                  "input_mask": tf.constant( input_mask, shape=[num_examples, seq_length], dtype=tf.int32),
                  "segment_ids": tf.zeros(shape=[num_examples, seq_length], dtype=tf.int32),}
                 ,tf.zeros(shape=[num_examples], dtype=tf.int32),))            

            d = d.batch(batch_size=batch_size, drop_remainder=drop_remainder)
            return d        

    return input_fn

In [41]:
def create_model(albert_config, mode, input_ids, input_mask, segment_ids,
                                 labels, num_labels):
    """Creates a classification model."""
    is_training = mode == tf.estimator.ModeKeys.TRAIN
    model = modeling.AlbertModel(
            config=albert_config,
            is_training=is_training,
            input_ids=input_ids,
            input_mask=input_mask,
            token_type_ids=segment_ids)

    #output_layer = model.get_sequence_output()
    output_layer = model.get_pooled_output()

    hidden_size = output_layer.shape[-1].value
    print("*************************num_labels***********", num_labels)

    output_weight = tf.get_variable(
            "output_weights", [num_labels, hidden_size],
            initializer=tf.truncated_normal_initializer(stddev=0.02))

    output_bias = tf.get_variable(
            "output_bias", [num_labels], initializer=tf.zeros_initializer())

    with tf.variable_scope("loss"):
        if mode == tf.estimator.ModeKeys.TRAIN:
            output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)
        output_layer = tf.reshape(output_layer, [-1, hidden_size])
        logits = tf.matmul(output_layer, output_weight, transpose_b=True)
        logits = tf.nn.bias_add(logits, output_bias)
        #logits = tf.reshape(logits, [-1, MAX_LEN, num_labels])
        logits = tf.reshape(logits, [-1, num_labels])
        probabilities = tf.nn.softmax(logits, axis=-1)
        predict = tf.argmax(probabilities,axis=-1)        
        if ((mode == tf.estimator.ModeKeys.TRAIN)|(mode == tf.estimator.ModeKeys.EVAL)):
            log_probs = tf.nn.log_softmax(logits, axis=-1)
            one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)
            per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
            loss = tf.reduce_sum(per_example_loss)
            return (loss, per_example_loss, logits,predict)
        else: # Predict
            return (None, None, logits,predict)

In [42]:
def model_fn_builder(albert_config, num_labels, init_checkpoint, learning_rate,
                     num_train_steps, num_warmup_steps, use_tpu,
                     hub_module=None, optimizer="adamw"):
    """Returns `model_fn` closure for TPUEstimator."""

    def model_fn(features, labels, mode, params):	# pylint: disable=unused-argument
        """The `model_fn` for TPUEstimator."""

        tf.logging.info("*** Features ***")
        for name in sorted(features.keys()):
            tf.logging.info(f"	name = {name}, shape = {features[name].shape}")

        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        segment_ids = features["segment_ids"]
        label_ids = labels

        (total_loss, per_example_loss, logits, predictions) = \
                create_model(albert_config, mode, input_ids, input_mask,
                                         segment_ids, label_ids, num_labels)

        tvars = tf.trainable_variables()
        initialized_variable_names = {}
        scaffold_fn = None
        if init_checkpoint:
            (assignment_map, initialized_variable_names
            ) = modeling.get_assignment_map_from_checkpoint(tvars, init_checkpoint)
            if use_tpu:
                def tpu_scaffold():
                    tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
                    return tf.train.Scaffold()

                scaffold_fn = tpu_scaffold
            else:
                print("********weights is initialize from checkpoint**********")
                tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

        tf.logging.info("**** Trainable Variables ****")
        for var in tvars:
            init_string = ""
            if var.name in initialized_variable_names:
                init_string = ", *INIT_FROM_CKPT*"
            tf.logging.info(f"	name = {var.name}, shape = {var.shape}{init_string}")

        output_spec = None
        if mode == tf.estimator.ModeKeys.TRAIN:

            train_op = optimization.create_optimizer(
                    total_loss, learning_rate, num_train_steps, num_warmup_steps,
                    use_tpu, optimizer)

            output_spec = tf.contrib.tpu.TPUEstimatorSpec(
                    mode=mode,
                    loss=total_loss,
                    train_op=train_op,
                    scaffold_fn=scaffold_fn)
        elif mode == tf.estimator.ModeKeys.EVAL:
            def metric_fn(per_example_loss, label_ids, logits):
                predictions = tf.argmax(logits, axis=-1, output_type=tf.int32)
                accuracy = tf.metrics.mean(tf.math.equal(label_ids,predictions))
                loss = tf.metrics.mean(values=per_example_loss)
                #
                return {
                    "eval_accuracy":accuracy,
                    "eval_loss": loss,
                }                
            eval_metrics = (metric_fn, [per_example_loss, label_ids, logits])
            output_spec = tf.contrib.tpu.TPUEstimatorSpec(
                    mode=mode,
                    loss=total_loss,
                    eval_metrics=eval_metrics,
                    scaffold_fn=scaffold_fn)
        else: # Predict
            output_spec = tf.contrib.tpu.TPUEstimatorSpec(
                    mode=mode,
                    predictions= predictions,
                    scaffold_fn=scaffold_fn)
        return output_spec
    return model_fn

In [43]:
USE_TPU = False
model_fn = model_fn_builder(
        albert_config=config,
        num_labels=len(label_list),
        init_checkpoint=init_checkpoint,
        learning_rate=LEARNING_RATE,
        num_train_steps=num_train_steps,
        num_warmup_steps=NUM_WARMUP_STEPS,
        use_tpu=USE_TPU)

# If TPU is not available, this will fall back to normal Estimator on CPU
# or GPU.
estimator = tf.contrib.tpu.TPUEstimator(
        use_tpu=USE_TPU,
        model_fn=model_fn,
        config=run_config,
        train_batch_size=BATCH_SIZE,
        eval_batch_size=BATCH_SIZE,
        predict_batch_size=BATCH_SIZE)

INFO:tensorflow:Using config: {'_model_dir': 'finetune1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 900, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 60, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fdc3297d4e0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=2, num_shards=None, num_cores_per_replica=None, per_ho

In [ ]:
tf.logging.info("***** Running training *****")
# import sys
import time
start = time.time()  # 시작 시간 저장

train_input_fn = input_fn_builder(
        file = file_dir + "/train.csv",
        tokenizer = tokenizer,
        seq_length=MAX_LEN,
        drop_remainder=True)

estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

In [ ]:
predict_input_fn = input_fn_builder(
        file = file_dir + "/test.csv",
        tokenizer = tokenizer,
        seq_length=MAX_LEN,
        drop_remainder=False)


predictions = []
for prediction in estimator.predict(input_fn=predict_input_fn, checkpoint_path= OUTPUT_DIR + '/model.ckpt-50000'):
    #print("*******prediction*********", prediction)
    predictions.append(prediction)
    
# for prediction in estimator.predict(input_fn=predict_input_fn, checkpoint_path='animal/models/saved/model.ckpt-25258'):
#     #print("*******prediction*********", prediction)
#     predictions.append(prediction)    

In [12]:
df_test = pd.read_csv(file_dir + "/test.csv")
labels = list(df_test.label_id)
print(labels)
print(set(labels))
print(predictions)

[0, 5, 17, 17, 18, 17, 18, 18, 10, 10, 6, 16, 17, 16, 9, 18, 18, 18, 15, 17, 5, 5, 6, 17, 18, 17, 2, 18, 18, 16, 13, 13, 11, 7, 18, 6, 10, 13, 1, 14, 18, 18, 16, 17, 18, 11, 19, 5, 18, 18, 18, 15, 19, 8, 8, 0, 16, 9, 4, 16, 9, 15, 11, 18, 18, 4, 0, 16, 16, 18, 14, 19, 17, 16, 10, 2, 15, 19, 9, 17, 18, 12, 18, 9, 4, 17, 9, 18, 0, 10, 18, 16, 7, 18, 17, 17, 4, 18, 2, 14, 10, 2, 3, 18, 6, 4, 18, 5, 17, 18, 9, 18, 17, 17, 17, 18, 18, 1, 0, 11, 1, 3, 1, 13, 18, 3, 17, 18, 16, 1, 0, 13, 1, 18, 18, 17, 1, 16, 18, 18, 13, 5, 17, 1, 13, 18, 18, 18, 0, 6, 1, 18, 3, 19, 6, 1, 9, 17, 14, 18, 16, 10, 18, 4, 0, 17, 0, 18, 18, 18, 18, 18, 16, 14, 17, 10, 18, 16, 18, 18, 18, 13, 18, 1, 8, 18, 18, 17, 18, 16, 16, 6, 5, 13, 4, 18, 17, 9, 17, 5, 2, 0, 4, 18, 16, 4, 16, 9, 10, 18, 18, 11, 9, 17, 18, 15, 18, 1, 4, 18, 12, 17, 0, 17, 18, 16, 1, 10, 4, 18, 18, 18, 17, 6, 17, 9, 14, 18, 8, 4, 14, 1, 1, 8, 0, 18, 16, 4, 0, 17, 18, 18, 5, 7, 6, 5, 7, 17, 14, 13, 18, 7, 12, 16, 18, 17, 18, 16, 5, 17, 5, 3, 19, 1

In [13]:
df_train = pd.read_csv(file_dir + "/train.csv")
df_train.head(10)

,보호자명,환자명,품종코드,생일,성별코드,검사일,병원,진단코드,SE,label,label_id
0,강경민,마루,508.0,2016-12-04,M,2017-06-05,D,2236,"* 384,300원 수납하셨습니다 - 그림 CC : FPV 입원 현증...",others,18
1,조혜빈(R),모카,499.0,2016-08-16,FS,2017-08-16,F,2046,"cc ; 식도이물 서울숲 refer, 위내이물도 의심됨 보호자 돼지뼈를...",2046,2
2,홍경숙,사랑,125.0,2007-12-03,FS,2017-01-18,E,"normal,normal",$- S) V- 식욕 활기 백신 사상충 ...,normal,17
3,유차남R,사랑이,123.0,2012-02-27,F,2014-06-04,A,2185,40일전 분만 - 5마리. 우측 무릎 탈구.,2185,14
4,박연선,지코,598.0,2017-05-12,MN,2017-11-03,B,test,털이 푸석푸석하고 체중이 2kg 미만임 중성화 수술 보류. FIP Ab hign ...,test,19
5,정하나,롬멜,537.0,2015-10-11,MN,2017-07-20,F,"normal,normal",글로블린 수치가 높음 -> 염증성 질환이 의심됨 체온 39.4 ...,normal,17
6,김미옥,이슬이,131.0,2005-09-26,F,2016-04-04,E,"2101,2101","$- S) 어제부터 설사, 구토. 체온 높은 것 같다고. V- 식욕 활...",2101,9
7,이혜진(R),예동,119.0,2017-05-16,MN,2018-05-16,F,"normal,normal",cc : 금일 새벽 소파에서 뛰어내린 후 우측 앞다리 파행 방사선 : 요척골 골...,normal,17
8,김연숙(R),준,125.0,2006-05-01,M,2018-03-13,F,"normal,normal","cc : 아지병원에서 피하잠복고환 확인 후 refer, 평소 흡기시 코골이 방사선...",normal,17
9,김민정,오순이,598.0,2016-10-17,FS,2017-08-20,F,"normal,normal","Hx: 최근 발정 증상, 밤에 증상이 심해서 민원들어옴 PE: BAR 항체가 검...",normal,17


In [14]:
right = 0
wrong = 0
result_df = pd.DataFrame(columns=['label', 'predict'])

for i in range(len(labels)):
    if labels[i] == predictions[i] :
        right += 1
    else :
        wrong += 1
#         print( "Wrong label : {} predict {} ".format(labels[i], predictions[i]) )
    result_df.loc[i,'label'] = labels[i]
    result_df.loc[i,'predict'] = predictions[i]

accuracy = right/(right+wrong)
print("accuracy :", accuracy)
print("wrong {} right {} total {}".format(wrong, right, len(labels)))

accuracy : 0.514018691588785
wrong 468 right 495 total 963


In [15]:
LABEL = 16
print('Label :', LABEL)
print((result_df[result_df['label']==LABEL]).predict.value_counts())
temp_df = result_df[result_df['label']==LABEL]
print(len(temp_df[temp_df['predict']==LABEL])/len(temp_df))


Label : 16
16    58
18     6
17     5
1      3
15     1
13     1
9      1
2      1
Name: predict, dtype: int64
0.7631578947368421


In [16]:
(result_df[result_df['label']==1]).predict.value_counts()

1     20
2      6
18     5
5      3
17     2
3      2
19     1
11     1
4      1
16     1
Name: predict, dtype: int64

In [ ]:
import os 

predict_input_fn = input_fn_builder(
        file = file_dir + "/test.csv",
        tokenizer = tokenizer,
        seq_length=MAX_LEN,
        drop_remainder=False)

df_test = pd.read_csv(file_dir + "/test.csv")
labels = list(df_test.label_id)
# print(labels)
print(set(labels))
predictions = []

def make_prob (labels, predictions) :
    right = 0
    wrong = 0
    for i in range(len(labels)):
        if labels[i] == predictions[i] :
            right += 1
        else :
            wrong += 1
    accuracy = right/(right+wrong)
    return accuracy
            
    #         print( "Wrong label : {} predict {} ".format(labels[i], predictions[i]) )

saved_ckpts = [ckpt for ckpt in os.listdir(OUTPUT_DIR + '/') if ckpt.endswith('.index')] 
accuracy_df = pd.DataFrame(columns=['ckpt', 'accuracy'])
# print(saved_ckpts)

index = 0
for ckpt in saved_ckpts :
    ckpt = ckpt[:-6]
    print(ckpt)
    predictions = []
    for prediction in estimator.predict(input_fn=predict_input_fn, checkpoint_path=OUTPUT_DIR+'/' + ckpt):
        predictions.append(prediction)
    accuracy = make_prob(labels, predictions)
    accuracy_df.loc[index, 'ckpt'] = ckpt
    accuracy_df.loc[index, 'accuracy'] = accuracy
    index += 1
    
    

In [47]:
accuracy_df

,ckpt,accuracy
0,model.ckpt-60012,0.481828
1,model.ckpt-86755,0.464174
2,model.ckpt-11045,0.49325
3,model.ckpt-93444,0.49325
4,model.ckpt-88984,0.46729
5,model.ckpt-31044,0.469367
6,model.ckpt-33272,0.460021
7,model.ckpt-19905,0.491173
8,model.ckpt-75611,0.499481
9,model.ckpt-82296,0.476636


In [48]:
accuracy_df.sort_values(by=['ckpt'], axis=0)
accuracy_df['index'] = accuracy_df.ckpt.apply(lambda x : int(str(x).split('-')[1]) )
accuracy_df.sort_values(by=['index'], axis=0)

,ckpt,accuracy,index
13,model.ckpt-0,0.07892,0
24,model.ckpt-1474,0.4081,1474
17,model.ckpt-2950,0.500519,2950
22,model.ckpt-4425,0.472482,4425
12,model.ckpt-6630,0.454829,6630
39,model.ckpt-8833,0.475597,8833
2,model.ckpt-11045,0.49325,11045
32,model.ckpt-13240,0.495327,13240
41,model.ckpt-15454,0.488058,15454
36,model.ckpt-17677,0.475597,17677


In [4]:
!sleep 10s

In [5]:
print("hi")

hi
